In [1]:
import numpy as np
import pandas as pd 
pd.set_option("max_columns",None)
from sklearn.linear_model import LinearRegression, Ridge
#from sklearn.preprocessing import StandardScaler, PolynomialFeatures
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import mean_squared_error
#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.activations import relu,linear
#from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
#from matplotlib import units




 

import numpy as np 
import pandas as pd 
from math import sqrt

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime

from sklearn.model_selection import StratifiedShuffleSplit, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error,make_scorer,r2_score
from sklearn.inspection import plot_partial_dependence
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api  as sm
from scipy import stats
from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.linear_model import Lasso, Ridge, SGDRegressor,LinearRegression,RidgeCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler,RobustScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor,DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')
import pprint

pd.set_option('max_columns', 200)



In [ ]:
#df = pd.ExcelFile('C:/Users/halil/Desktop/soalr_data.xlsx').parse('sheet 1')
weather_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="weather")
guneko_raw = pd.read_excel('C:/Users/halil/Desktop/soalr_data.xlsx',sheet_name="1000255-GUNEKO")

In [ ]:
guneko_production = guneko_raw[["Date","Production"]]
guneko_gti = guneko_raw[["Date.1","GTI"]]


weather_guneko = weather_raw.loc[weather_raw.name ==1000255]
dataset1 = pd.merge(guneko_production,weather_guneko,left_on="Date",right_on="date")
#dataset1 = ali.drop(["name","date","lat","lon"],axis=1)

In [ ]:
y  = dataset1["Production"]
x = dataset1.drop("Production",axis=1)

In [ ]:
#train test split

x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.8,shuffle=False,random_state=1)

In [ ]:
def onehot(dataset):
    veli = dataset.copy()
    dataset= dataset.set_index(["Date"])
    year = pd.DataFrame(data=pd.get_dummies(dataset.index.year,prefix="year"))
    month = pd.DataFrame(data=pd.get_dummies(dataset.index.month, prefix="month"))
    day = pd.DataFrame(data=pd.get_dummies(dataset.index.day,prefix="day"))
    hour = pd.DataFrame(data=pd.get_dummies(dataset.index.hour,prefix="hour"))
    frames = [year,month,day,hour]

    final = veli.join(frames)
    final = final.drop(["Date","name","date","lat","lon"],axis=1)

    return final

    


In [ ]:
def normalize(df,column_list):
    for col in column_list:
        feature_range = (-1,1)
        min_max_scaler = MinMaxScaler(feature_range=feature_range)

        df[col] = min_max_scaler.fit_transform(df[col].values.reshape(-1,1))

    return df

In [ ]:
x_train = onehot(x_train)
x_test = onehot(x_test)

In [ ]:
x_train = normalize(x_train,x_train.columns[0:10])
x_test = normalize(x_test,x_test.columns[0:10])


In [ ]:
min_max_scaler = MinMaxScaler(feature_range=(-1,1))
y_train = min_max_scaler.fit_transform(y_train.values.reshape(-1,1))
y_test = min_max_scaler.fit_transform(y_test.values.reshape(-1,1))

In [ ]:
df = pd.DataFrame(y_train, columns = ['production'])

aly = df["production"]
aly

0       -1.000000
1       -1.000000
2       -1.000000
3       -1.000000
4       -1.000000
           ...   
15223    0.647648
15224    0.725726
15225    0.673674
15226    0.501502
15227    0.191191
Name: production, Length: 15228, dtype: float64

In [ ]:
size = int(len(normalized) * 0.8)
train_dataset = normalized[:size]
test_dataset = normalized[size:]


In [130]:
%%time
clfs = []

seed = 3

clfs.append(("LinearRegression", Pipeline([("Scaler", RobustScaler()), ("LogReg", LinearRegression())])))

clfs.append(("XGB", Pipeline([("Scaler", RobustScaler()), ("XGB", XGBRegressor())]))) 

clfs.append(("KNN", Pipeline([("Scaler", RobustScaler()),("KNN", KNeighborsRegressor())]))) 

clfs.append(("DTR", Pipeline([("Scaler", RobustScaler()), ("DecisionTrees", DecisionTreeRegressor())]))) 

clfs.append(("RFRegressor", Pipeline([("Scaler", RobustScaler()), ("RandomForest", RandomForestRegressor())]))) 

clfs.append(("GBRegressor", Pipeline([("Scaler", RobustScaler()), ("GradientBoosting", GradientBoostingRegressor())]))) 

clfs.append(("MLP", Pipeline([("Scaler", RobustScaler()),("MLP Regressor", MLPRegressor())])))

clfs.append(("EXT Regressor",Pipeline([("Scaler", RobustScaler()),("ExtraTrees", ExtraTreeRegressor())])))

clfs.append(("SV Regressor", Pipeline([("Scaler", RobustScaler()),("ExtraTrees", SVR())])))

scoring = 'r2'
n_folds = 10
msgs = []
results, names  = [], [] 

for name, model  in clfs:
    kfold = KFold(n_splits=n_folds, random_state=None)
    cv_results = cross_val_score(model, x_train, y_train,cv=kfold, scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(), cv_results.std())
    msgs.append(msg)
    print(msg)

LinearRegression: -1865878302575291793408.000000 (+/- 5597634907725874855936.000000)
XGB: 0.881599 (+/- 0.061454)
KNN: 0.854822 (+/- 0.094151)
DTR: 0.800709 (+/- 0.107840)
RFRegressor: 0.881784 (+/- 0.071600)
GBRegressor: 0.887422 (+/- 0.060893)
MLP: 0.797446 (+/- 0.096995)
EXT Regressor: 0.809056 (+/- 0.108848)
SV Regressor: 0.873120 (+/- 0.077976)
Wall time: 1min 56s


In [ ]:
%%time
clfs = []
seed = 3

clfs.append(("LinearRegression", Pipeline([("LogReg", LinearRegression())])))

clfs.append(("XGB", Pipeline([("XGB", XGBRegressor())]))) 

clfs.append(("KNN", Pipeline([("KNN", KNeighborsRegressor())]))) 

clfs.append(("DTR", Pipeline([("DecisionTrees", DecisionTreeRegressor())]))) 

clfs.append(("RFRegressor", Pipeline([("RandomForest", RandomForestRegressor())]))) 

clfs.append(("GBRegressor", Pipeline([("GradientBoosting", GradientBoostingRegressor())]))) 

clfs.append(("MLP", Pipeline([("MLP Regressor", MLPRegressor())])))

clfs.append(("EXT Regressor",Pipeline([("ExtraTrees", ExtraTreeRegressor())])))

clfs.append(("SV Regressor", Pipeline([("ExtraTrees", SVR())])))

scoring = 'mae'
n_folds = 10
msgs = []
results, names  = [], [] 

for name, model  in clfs:
    kfold = KFold(n_splits=n_folds, random_state=None)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    msg = "%s: %f (+/- %f)" % (name, cv_results.mean(), cv_results.std())
    msgs.append(msg)
    print(msg)